In [1]:
%matplotlib inline
import numpy as np

import jupyter_manim
from manim import *

Manim Community v0.13.1

In [33]:
%%manim -qm -v WARNING qftcircuit

class qftcircuit(Scene):
    config= {
        "x_lim":int,    # define leftmost lowerlimit of x for circuit and gates
        "y_lim":int,    # define up-most limit of y for circuit and gates
        
    }
    def __init__(self, **kwargs):
        super(qftcircuit, self).__init__()       #inheritance from class Scene
        self.x_lim=-5
        self.y_lim=-2
        self.gap=1
        self.n_qubit=4
        self.x_range=[None for i in range(self.n_qubit+1)]
        self.dots=[Dot() for i in range(self.n_qubit)]
        self.circuitline=[[None for i in range(5)] for i in range(5)]   # List to hold all circuit line elements and VGroup  to include all the Mobjects of circuitline
        self.circuit=VGroup()
    

    def construct(self):
        
        # Algorithm name
        algorithm="Quantum Fourier Transform"
        title = Title(algorithm,include_underline=False).set_y(3.1)     #title of the ALgorithm
        self.add_foreground_mobject(title)

        #section zero: create all registers: quantum and classical
        # regs=[MathTex(r"q_"+str(i)) for i in range(n_qubit)] # create quantum register
        
        registers=VGroup()
        regs=[MathTex(r"q_"+str(i)) for i in range(self.n_qubit)]
        for i,reg in enumerate(regs):       #add registers to the scene
            reg.set_x(self.x_lim).set_y(self.y_lim+i*self.gap)
            registers.add(reg)
        self.add(registers)

        def create_gate(gate:str,x:int,y:int,control=None,scale=1,textcolor=WHITE,boxcolor=BLUE,fontsize=48):
            if control:
                dot=Dot().set_x(x).set_y(control)
                control_line= Line([x,control,0], [x,y,0])
                gate_ = MathTex(gate,color=textcolor,font_size=fontsize).scale(scale+0.5).set_x(x).set_y(y)
                gate_.bg = BackgroundRectangle(gate_,stroke_width=1,color=boxcolor, fill_opacity=1).scale(scale+1)
                return VGroup(control_line,gate_.bg,gate_,dot)
            else:
                gate_ = MathTex(gate,color=textcolor,font_size=fontsize).scale(scale+0.5).set_x(x).set_y(y)
                gate_.bg = BackgroundRectangle(gate_,stroke_width=1,color=boxcolor, fill_opacity=1).scale(scale+1)
                return VGroup(gate_.bg,gate_)
        
        def draw_circuitline(sect:int,xrange:list,ydiff=self.gap,barrier=True):
            for i in range(self.n_qubit):
                self.circuitline[sect][i]=Line([xrange[0],ydiff-i,-0.5], [xrange[1],ydiff-i,-0.5])
                self.circuit.add(self.circuitline[sect][i])
                self.add(self.circuitline[sect][i])
            if barrier:
                barrierline=DashedLine([xrange[1],self.y_lim-0.5,0], [xrange[1],self.y_lim+self.gap*(self.n_qubit-0.5),0],dashed_ratio=0.75).set_opacity(0.5)
                self.add(barrierline)
                # self.add(VGroup(circuit,barrierline))
                # return VGroup(circuit,barrier)
        def move_dot(sect: int,dots=self.dots):
            animation=[]
            for j in range(self.n_qubit):
                animation.append(MoveAlongPath(dots[j], self.circuitline[sect][j]))
            # self.play(*animation)
            return animation

        def qft_element(sect=1,ctr_rep=0,barrier=True):
            
            self.x_range[0]=[self.x_lim+0.5,self.x_lim+1.5+0.75*(ctr_rep)]
            if sect>0:
                self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+1+0.5*(ctr_rep)]
            sect-=1
            
            draw_circuitline(sect,xrange=self.x_range[sect],barrier=barrier)
            hgate=create_gate(r"H",x=1.01*self.x_range[sect][0]+0.5,y=self.y_lim+sect*self.gap,scale=0.1*self.n_qubit)
            phaseanim=[]
            for i in range(ctr_rep):
                i+=1
                angle=int(2**(i))
                phasegate=create_gate(r"\pi"+"/"+str(angle),x=self.x_range[sect][0]+(i+0.5)*0.75,y=self.y_lim+(sect+0.01)*self.gap,
                                        scale=0.03*self.n_qubit,fontsize=48,control=self.y_lim+(sect+i+0.01)*self.gap)
                            
                phaseanim.append(Create(phasegate))
            if ctr_rep>0:
                self.play(Create(hgate),*phaseanim,runtime=2*ctr_rep)
            else:
                self.play(Create(hgate),runtime=1)
            print(self.x_range)
            

    
        # Section I
        qft_element(sect=1,ctr_rep=3)

        texTemplate = TexTemplate()
        texTemplate.add_to_preamble(r"\usepackage{braket}")
       
        state_1a= MathTex(r"\ket{q_0}",tex_template=texTemplate, 
                                        font_size=20).set_x(self.x_range[0][0]).set_y(self.y_lim+0.5+3*self.gap)
        state_2a= MathTex(r"\ket{q_1}",tex_template=texTemplate, 
                                        font_size=20).set_x(self.x_range[0][0]).set_y(self.y_lim+0.5+2*self.gap)
        state_3a= MathTex(r"\ket{q_2}}",tex_template=texTemplate, 
                                        font_size=20).set_x(self.x_range[0][0]).set_y(self.y_lim+0.5+self.gap)
        state_4a= MathTex(r"\ket{q_3}}",tex_template=texTemplate, 
                                        font_size=20).set_x(self.x_range[0][0]).set_y(self.y_lim+0.5)
        self.play(Create(state_1a),Create(state_2a),Create(state_3a),Create(state_4a))
        self.wait()

        # state_1b= MathTex(r"\ket{q_0}",tex_template=texTemplate, 
        #                                 font_size=20).set_x(self.x_range[1][0]).set_y(self.y_lim+0.5+3*self.gap)
        # state_2b= MathTex(r"\ket{q_1}",tex_template=texTemplate, 
        #                                 font_size=20).set_x(self.x_range[1][0]).set_y(self.y_lim+0.5+2*self.gap)
        # state_3b= MathTex(r"\ket{q_2}}",tex_template=texTemplate, 
        #                                 font_size=20).set_x(self.x_range[1][0]).set_y(self.y_lim+0.5+self.gap)
        state_4b= MathTex(r"\frac{1}{\sqrt{2}} \bigr( \ket{0} + e^{2\pi i 0.q_{3}q_{2}q_{1}q_{0} } \ket{1}\bigr)",
                            tex_template=texTemplate, font_size=20).set_x(self.x_range[1][0]).set_y(self.y_lim+0.5)

        self.play(state_1a.animate.move_to([self.x_range[1][0],self.y_lim+0.5+3*self.gap,0]),
                    state_2a.animate.move_to([self.x_range[1][0],self.y_lim+0.5+2*self.gap,0]),
                    state_3a.animate.move_to([self.x_range[1][0],self.y_lim+0.5+self.gap,0]),
                    FadeTransform(state_4a,state_4b),
                    *move_dot(0),runtime=2)

        
        
        # self.play(state_psi.animate.move_to([sect_range1[1],2.5,0]),
        #             FadeTransform(state_1a,state_1b),
        #             *move_dot(sect=1),
        #             runtime=3)

        # Section II
        qft_element(sect=2,ctr_rep=2)
        sect=2
        state_3c= MathTex(r"\frac{1}{\sqrt{2}} \bigr( \ket{0} + e^{2\pi i 0.q_{3}q_{2}q_{1}q_{0} } \ket{1}\bigr)",
                            tex_template=texTemplate, font_size=20).set_x(self.x_range[sect][0]).set_y(self.y_lim+0.5+self.gap)
        self.play(state_1a.animate.move_to([self.x_range[sect][0],self.y_lim+0.5+3*self.gap,0]),
                    state_2a.animate.move_to([self.x_range[sect][0],self.y_lim+0.5+2*self.gap,0]),
                    FadeTransform(state_3a,state_3c),
                    state_4b.animate.move_to([self.x_range[sect][0],self.y_lim+0.5,0]),
                    *move_dot(1),runtime=2)
        # Section III
        qft_element(sect=3,ctr_rep=1)
        sect=3
        state_2d= MathTex(r"\frac{1}{\sqrt{2}} \bigr( \ket{0} + e^{2\pi i 0.q_{3}q_{2} } \ket{1}\bigr)",
                            tex_template=texTemplate, font_size=20).set_x(self.x_range[sect][0]).set_y(self.y_lim+0.5+2*self.gap)
        self.play(state_1a.animate.move_to([self.x_range[sect][0],self.y_lim+0.5+3*self.gap,0]),
                    FadeTransform(state_2a,state_2d),
                    state_3c.animate.move_to([self.x_range[sect][0],self.y_lim+0.5+self.gap,0]),
                    state_4b.animate.move_to([self.x_range[sect][0],self.y_lim+0.5,0]),
                    *move_dot(2),runtime=2)
        # #section IV
        qft_element(sect=4,ctr_rep=0,barrier=False)
        sect=4
        state_1e= MathTex(r"\frac{1}{\sqrt{2}} \bigr( \ket{0} + e^{2\pi i 0.q_{3} } \ket{1}\bigr)",
                            tex_template=texTemplate, font_size=20).set_x(self.x_range[sect][0]).set_y(self.y_lim+0.5+3*self.gap)
        self.play(FadeTransform(state_1a,state_1e),
                    state_2d.animate.move_to([self.x_range[sect][0],self.y_lim+0.5+2*self.gap,0]),
                    state_3c.animate.move_to([self.x_range[sect][0],self.y_lim+0.5+self.gap,0]),
                    state_4b.animate.move_to([self.x_range[sect][0],self.y_lim+0.5,0]),
                    *move_dot(3),runtime=2)
        self.wait(2)
        




[[-4.5, -1.25], [-1.25, 1.25], None, None, None]


[[-4.5, -2.0], [-1.25, 1.25], [1.25, 3.25], None, None]


[[-4.5, -2.75], [-1.25, 1.25], [1.25, 3.25], [3.25, 4.75], None]


[[-4.5, -3.5], [-1.25, 1.25], [1.25, 3.25], [3.25, 4.75], [4.75, 5.75]]


In [8]:
import manim as manim
manim.__version__

'0.13.1'